<a href="https://colab.research.google.com/github/jjennings955/Neural-Network-Notebooks/blob/master/Backprop_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sympy import *

def custom_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return printing.latex(exp,**options)
  
#sp.init_session(use_latex='mathjax', latex_printer=custom_latex_printer)
init_printing(use_latex='mathjax', latex_printer=custom_latex_printer)

# Deriving Softmax Jacobian using sympy

In [0]:
x_1, x_2, x_3 = symbols('x_1, x_2, x_3') # declare symbols
x_m = Matrix([[x_1, x_2, x_3]]).T

In [3]:
exp_x = x_m .applyfunc(exp) # exponentiate element-wise
exp_x

⎡ x₁⎤
⎢ℯ  ⎥
⎢   ⎥
⎢ x₂⎥
⎢ℯ  ⎥
⎢   ⎥
⎢ x₃⎥
⎣ℯ  ⎦

In [4]:
denom = sum(exp_x) # compute the denominator (sum of exponentiation)
denom

 x₁    x₂    x₃
ℯ   + ℯ   + ℯ  

In [5]:
softmax = exp_x / denom # softmax is exp(x)/sum(exp(x))
softmax

⎡       x₁      ⎤
⎢      ℯ        ⎥
⎢───────────────⎥
⎢ x₁    x₂    x₃⎥
⎢ℯ   + ℯ   + ℯ  ⎥
⎢               ⎥
⎢       x₂      ⎥
⎢      ℯ        ⎥
⎢───────────────⎥
⎢ x₁    x₂    x₃⎥
⎢ℯ   + ℯ   + ℯ  ⎥
⎢               ⎥
⎢       x₃      ⎥
⎢      ℯ        ⎥
⎢───────────────⎥
⎢ x₁    x₂    x₃⎥
⎣ℯ   + ℯ   + ℯ  ⎦

# Softmax Jacobian


In [6]:
softmax.jacobian(x_m)

⎡       x₁                2⋅x₁                        x₁  x₂                  
⎢      ℯ                 ℯ                          -ℯ  ⋅ℯ                    
⎢─────────────── - ──────────────────           ──────────────────            
⎢ x₁    x₂    x₃                    2                            2            
⎢ℯ   + ℯ   + ℯ     ⎛ x₁    x₂    x₃⎞            ⎛ x₁    x₂    x₃⎞             
⎢                  ⎝ℯ   + ℯ   + ℯ  ⎠            ⎝ℯ   + ℯ   + ℯ  ⎠             
⎢                                                                             
⎢               x₁  x₂                        x₂                2⋅x₂          
⎢             -ℯ  ⋅ℯ                         ℯ                 ℯ              
⎢         ──────────────────           ─────────────── - ──────────────────   
⎢                          2            x₁    x₂    x₃                    2   
⎢         ⎛ x₁    x₂    x₃⎞            ℯ   + ℯ   + ℯ     ⎛ x₁    x₂    x₃⎞    
⎢         ⎝ℯ   + ℯ   + ℯ  ⎠                         

# How to craft jacobian as function of the softmax outputs

In [7]:
diag(*softmax) - softmax*softmax.T

⎡       x₁                2⋅x₁                        x₁  x₂                  
⎢      ℯ                 ℯ                          -ℯ  ⋅ℯ                    
⎢─────────────── - ──────────────────           ──────────────────            
⎢ x₁    x₂    x₃                    2                            2            
⎢ℯ   + ℯ   + ℯ     ⎛ x₁    x₂    x₃⎞            ⎛ x₁    x₂    x₃⎞             
⎢                  ⎝ℯ   + ℯ   + ℯ  ⎠            ⎝ℯ   + ℯ   + ℯ  ⎠             
⎢                                                                             
⎢               x₁  x₂                        x₂                2⋅x₂          
⎢             -ℯ  ⋅ℯ                         ℯ                 ℯ              
⎢         ──────────────────           ─────────────── - ──────────────────   
⎢                          2            x₁    x₂    x₃                    2   
⎢         ⎛ x₁    x₂    x₃⎞            ℯ   + ℯ   + ℯ     ⎛ x₁    x₂    x₃⎞    
⎢         ⎝ℯ   + ℯ   + ℯ  ⎠                         

In [0]:
import numpy as np

# Backpropagation example

# Set up our input and targets

In [9]:
x = np.float32([[0.2, -0.3]]).T
y = np.float32([[0.0, 0.0, 1]]).T
print(x)
print(y)

[[ 0.2]
 [-0.3]]
[[0.]
 [0.]
 [1.]]


# Set up weights/biases of first layer

In [10]:
W_1 = np.float32([[1, 0.2], [-1, 3.0], [0.5, 0.5]])
b_1 = np.float32([[0.1, 0.2, 0]]).T
print(W_1)
print(b_1)

[[ 1.   0.2]
 [-1.   3. ]
 [ 0.5  0.5]]
[[0.1]
 [0.2]
 [0. ]]


# Set up weights/biases of second layer


In [11]:
W_2 = np.float32([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.2, -0.2, 0.1]])
b_2 = np.float32([[0, 0.5, 0]]).T
print(W_2)
print(b_2)

[[ 0.1  0.2  0.3]
 [ 0.4  0.5  0.6]
 [ 0.2 -0.2  0.1]]
[[0. ]
 [0.5]
 [0. ]]


# Declare functions for computing sigmoid, softmax, and cross-entropy loss

In [0]:

def sigmoid(x):
  return 1/(1 + np.exp(-x))

def softmax(x):
  e_x = np.exp(x)
  denom = np.sum(e_x)
  return e_x / denom

def xent(y, y_hat):
  return -np.sum(y * np.log(y_hat))
  

# Forward Pass

# Compute first layer output

In [13]:
net_1 = np.dot(W_1, x) + b_1
print(net_1)
z_1 = sigmoid(net_1)
print(z_1)

[[ 0.24000001]
 [-0.90000004]
 [-0.05      ]]
[[0.55971366]
 [0.2890505 ]
 [0.48750263]]


# Compute second layer output

In [14]:
net_2 = np.dot(W_2, z_1) + b_2
print(net_2)
y_hat = softmax(net_2)
print(y_hat)

[[0.26003227]
 [1.1609123 ]
 [0.10288289]]
[[0.23167749]
 [0.57033634]
 [0.19798611]]


# Compute categorical cross-entropy loss

In [15]:
L = xent(y, y_hat)
print(L)

1.6195585


# Backward Pass

In [16]:

# dL_dyhat = np.float32([0, 0, -1/y_hat[2]]) # this is hard-coded, should be based on y
dL_dyhat = (-y*(1/y_hat)).T # jacobian of L with respect to y_hat
g_1 = dL_dyhat.T * 1
print(g_1)

[[-0.       ]
 [-0.       ]
 [-5.0508595]]


In [17]:
# compute Jacobian of softmax with respect to inputs
dy_hat_dnet2 = np.diagflat(y_hat) - np.dot(y_hat, y_hat.T)
print(dy_hat_dnet2)

[[ 0.17800303 -0.1321341  -0.04586893]
 [-0.1321341   0.24505278 -0.11291867]
 [-0.04586893 -0.11291867  0.15878761]]


In [18]:
g_2 = np.dot(dy_hat_dnet2.T, g_1)
print(g_2)

[[ 0.2316775 ]
 [ 0.57033634]
 [-0.8020139 ]]


In [19]:
dL_b2 = g_2 # backpropagation through addition is just the identity function
print(dL_b2)

[[ 0.2316775 ]
 [ 0.57033634]
 [-0.8020139 ]]


In [20]:
dL_W2 = g_2 * z_1.T # backpropagation through matrix-vector product, wrt W2 using jacobian-vector product trick
dL_W2

array([[ 0.12967306,  0.0669665 ,  0.1129434 ],
       [ 0.31922504,  0.164856  ,  0.27804047],
       [-0.44889814, -0.2318225 , -0.39098388]], dtype=float32)

In [0]:
dL_dz1 = np.dot(W_2.T, g_2)  # backpropagation through matrix-vector product, wrt input z_1 using jacobian-vector product trick

In [0]:
g_3 = dL_dnet1 = z_1 * (1 - z_1) * g_2 # backpropagation through sigmoid, using jacobian-vector product trick for elementwise functions

In [0]:
dL_db1 = g_3 # backpropagation through addition is just the identity function

In [29]:
print(g_3)
print(x)

[[ 0.05709328]
 [ 0.1172043 ]
 [-0.20037821]]
[[ 0.2]
 [-0.3]]


In [30]:
dL_W1 = np.dot(g_3, x.T)  # backpropagation through matrix-vector product, wrt input W_1 using jacobian-vector product trick
print(dL_W1)

[[ 0.01141866 -0.01712798]
 [ 0.02344086 -0.03516129]
 [-0.04007564  0.06011346]]


In [0]:
alpha = 1.0

# Perform gradient descent update

In [0]:
W_1new = W_1 - alpha*dL_W1
b_1new = b_1 - alpha*dL_db1
W_2new = W_2 - alpha*dL_W2
b_2new = b_2 - alpha*dL_b2


In [33]:
print('old output', y_hat)
new_output = softmax(np.dot(W_2new, sigmoid(np.dot(W_1new, x) + b_1new) + b_2new))
print('new output', new_output)

old output [[0.23167749]
 [0.57033634]
 [0.19798611]]
new output [[0.24301268]
 [0.31357023]
 [0.4434171 ]]


In [0]:
# We can see the weights were updated to make the target class (class 3) increase

# Deriving Jacobian Vector Product for Matrix-Vector product

In [35]:
# Weight matrix
w_syms = MatrixSymbol('W', 3,3)
W = Matrix(w_syms)
W

⎡W₀₀  W₀₁  W₀₂⎤
⎢             ⎥
⎢W₁₀  W₁₁  W₁₂⎥
⎢             ⎥
⎣W₂₀  W₂₁  W₂₂⎦

In [36]:
# input
x_syms = MatrixSymbol('x', 3,1)
x = Matrix(x_syms)
x

⎡x₀₀⎤
⎢   ⎥
⎢x₁₀⎥
⎢   ⎥
⎣x₂₀⎦

In [37]:
# gradient to back-propagate
g_syms = MatrixSymbol('g', 3,1)
g = Matrix(g_syms)
g

⎡g₀₀⎤
⎢   ⎥
⎢g₁₀⎥
⎢   ⎥
⎣g₂₀⎦

In [38]:
# Jacobian-vector product with respect to W
# notice the annoying shape
J = (W*x).jacobian(W.reshape(9, 1)).T * g
J

⎡g₀₀⋅x₀₀⎤
⎢       ⎥
⎢g₀₀⋅x₁₀⎥
⎢       ⎥
⎢g₀₀⋅x₂₀⎥
⎢       ⎥
⎢g₁₀⋅x₀₀⎥
⎢       ⎥
⎢g₁₀⋅x₁₀⎥
⎢       ⎥
⎢g₁₀⋅x₂₀⎥
⎢       ⎥
⎢g₂₀⋅x₀₀⎥
⎢       ⎥
⎢g₂₀⋅x₁₀⎥
⎢       ⎥
⎣g₂₀⋅x₂₀⎦

In [39]:
# We would have to reshape it to this
J.reshape(3,3)

⎡g₀₀⋅x₀₀  g₀₀⋅x₁₀  g₀₀⋅x₂₀⎤
⎢                         ⎥
⎢g₁₀⋅x₀₀  g₁₀⋅x₁₀  g₁₀⋅x₂₀⎥
⎢                         ⎥
⎣g₂₀⋅x₀₀  g₂₀⋅x₁₀  g₂₀⋅x₂₀⎦

# Or we can do it like this

In [40]:
g * x.T

⎡g₀₀⋅x₀₀  g₀₀⋅x₁₀  g₀₀⋅x₂₀⎤
⎢                         ⎥
⎢g₁₀⋅x₀₀  g₁₀⋅x₁₀  g₁₀⋅x₂₀⎥
⎢                         ⎥
⎣g₂₀⋅x₀₀  g₂₀⋅x₁₀  g₂₀⋅x₂₀⎦